In [3]:
import json
import pandas as pd

from collections import defaultdict, Counter
from abandabot.evaluate import evaluate_performance

In [4]:
with open("reports/performance.json") as f:
    perf = json.load(f)

In [5]:
table = []
for setting, perf_logs in perf.items():
    model = setting.split("+")[0]
    context = "+".join(setting.split("+")[1:])
    avg_perf = pd.DataFrame(perf_logs).mean()
    std_pref = pd.DataFrame(perf_logs).std()
    results = {"model": model, "context": context}
    for key in avg_perf.keys():
        results[key] = f"{avg_perf[key]:.3f}±{std_pref[key]:.3f}"
    table.append(results)

ground_truth = pd.read_csv("ground_truth.csv")
ground_truth = ground_truth[ground_truth.impactful.isin(("Yes", "No"))]

table.append(
    {
        "model": "yesman",
        "context": "yesman",
        **evaluate_performance(
            ground_truth.impactful, ["Yes"] * len(ground_truth), "Yes", "No"
        ),
    }
)

table.append(
    {
        "model": "noman",
        "context": "noman",
        **evaluate_performance(
            ground_truth.impactful, ["No"] * len(ground_truth), "Yes", "No"
        ),
    }
)

table.append(
    {
        "model": "random",
        "context": "random",
        "errors": 0,
        "macro_precision": 0.5,
        "macro_recall": 0.5,
        "macro_f1": 0.5,
    }
)

table = pd.DataFrame(table)
table["model context macro_precision macro_recall macro_f1".split()]

,model,context,macro_precision,macro_recall,macro_f1
0,gpt-4o,reasoning,0.739±0.039,0.618±0.027,0.623±0.036
1,gpt-4o,reasoning+theory,0.709±0.069,0.601±0.047,0.599±0.065
2,gpt-4o,context+reasoning,0.672±0.037,0.657±0.039,0.661±0.039
3,gpt-4o,context+reasoning+theory,0.743±0.047,0.752±0.045,0.746±0.046
4,deepseek-v3,reasoning,0.753±0.049,0.581±0.025,0.566±0.038
5,deepseek-v3,reasoning+theory,0.772±0.094,0.625±0.052,0.632±0.065
6,deepseek-v3,context+reasoning,0.641±0.032,0.655±0.036,0.643±0.034
7,deepseek-v3,context+reasoning+theory,0.739±0.027,0.772±0.030,0.746±0.029
8,llama-v3p3,reasoning,0.798±0.099,0.541±0.020,0.495±0.035
9,llama-v3p3,reasoning+theory,0.734±0.058,0.603±0.029,0.602±0.038


In [8]:
table[table.context == "context+reasoning+theory"]

,model,context,errors,yes_precision,yes_recall,yes_f1,no_precision,no_recall,no_f1,macro_precision,macro_recall,macro_f1
3,gpt-4o,context+reasoning+theory,1.200±1.398,0.857±0.029,0.825±0.045,0.840±0.031,0.628±0.072,0.679±0.072,0.651±0.062,0.743±0.047,0.752±0.045,0.746±0.046
7,deepseek-v3,context+reasoning+theory,0.000±0.000,0.889±0.022,0.760±0.031,0.819±0.023,0.590±0.037,0.784±0.047,0.672±0.036,0.739±0.027,0.772±0.030,0.746±0.029
11,llama-v3p3,context+reasoning+theory,0.000±0.000,0.789±0.018,0.912±0.022,0.846±0.018,0.690±0.064,0.444±0.051,0.540±0.054,0.740±0.040,0.678±0.032,0.693±0.035
15,gemini-2.0,context+reasoning+theory,0.000±0.000,0.755±0.013,0.977±0.020,0.852±0.013,0.854±0.122,0.276±0.048,0.414±0.060,0.805±0.064,0.627±0.027,0.633±0.035
